In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.abspath(".."))

import gym
import random
import argparse
import numpy as np

import torch
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

from src.agent import AgentA2C
from src.utils import TrajStats, set_seeds

In [2]:
def compute_kl(logits_1, logits_2):
    """
    Computes KL divergence between discrete distributions
    """
    
    probs_1 = F.softmax(logits_1, dim=-1)
    kl_components = probs_1 * (F.log_softmax(logits_1, dim=-1) - F.log_softmax(logits_2, dim=-1))
    return torch.mean(torch.sum(kl_components, dim=0))


def get_flat_params(model):
    return torch.cat([param.data.view(-1) for param in model.parameters()])


def set_flat_params(model, flat_params):
    ind_start = 0
    for param in model.parameters():
        ind_end = ind_start + np.prod(param.shape)
        param.data.copy_(flat_params[ind_start : ind_end].view(param.shape))
        ind_start = ind_end
        
        
def get_flat_grads(model, loss):
    """
    Walkaround for computing grads in case loss does not depend on some leafs
    TODO: remove `try` later
    """
    
    grads = []
    #for name, param in model.named_parameters():
    for param in model.parameters():
        try:
            grads.append(torch.autograd.grad(loss, param, create_graph=True)[0])
        except RuntimeError:
            grads.append(torch.zeros_like(param))
            
    return torch.cat([grad.view(-1) for grad in grads])
        
        
def cg(matvec, b, cg_iters=10, residual_tol=1e-10):
    """
    Solves system Ax=b via conjugate gradients method.
    Adapted from John Schulman's code:
    https://github.com/joschu/modular_rl/blob/master/modular_rl/trpo.py
    Arguments:
        matvec        --  matrix-vector product function
        b             --  right-hand side
        cg_iters      --  number of iterations
        residual_tol  --  tolerance
    """
    
    x = torch.zeros(b.size())
    r = b.clone()
    p = b.clone()
    rdotr = torch.dot(r, r)
    
    for i in range(cg_iters):
        Ap = matvec(p)
        alpha = rdotr / torch.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        newrdotr = torch.dot(r, r)
        beta = newrdotr / rdotr
        p = r + beta * p
        rdotr = newrdotr
        if rdotr < residual_tol:
            break
            
    return x

def linesearch(f, x, fullstep, expected_improve_rate, max_backtracks=10, accept_ratio=.1):
    """
    Backtracking linesearch for finding optimal proposed step size.
    Adapted from John Schulman's code:
    https://github.com/joschu/modular_rl/blob/master/modular_rl/trpo.py
    Arguments:
        f                      --  
        x                      --  
        fullstep               --  
        expected_improve_rate  --  
        max_backtracks         --
        accept_ratio           --
    """
    fval = f(x)
    for stepfrac in .5**np.arange(max_backtracks):
        xnew = x + stepfrac*fullstep
        newfval = f(xnew)
        actual_improve = (fval - newfval).data[0]
        expected_improve = expected_improve_rate*stepfrac
        ratio = actual_improve/expected_improve
        if ratio > accept_ratio and actual_improve > 0:
            return True, xnew
    return False, x


def hess_vec_full(vec, model, grads, damping):
    grads_vec = torch.dot(grads, Variable(vec))
    res = get_flat_grads(model, grads_vec).data
    return res + damping * vec


def compute_obj_full(flat_params, agent, ts, gamma, lambda_gae):
    # TODO: rewrite in vector form later and probably without new TrajStats 
    # TODO: and probably rewrite TrajStats, e.g. append to dict, make calc_gae not a method of class
    
    set_flat_params(agent, flat_params)
    cur_ts = TrajStats()
    for s, a, r in ts.get_sar():
        logits, value = agent.forward(s)
        cur_ts.append(r, F.log_softmax(logits, dim=-1)[a], value, logits, s, a)
        
    old_logs_pi = ts.get_logs_pi_a()
    advantages = cur_ts.calc_gaes(gamma, lambda_gae)
    logs_pi = cur_ts.get_logs_pi_a()
    return -(torch.exp(logs_pi - old_logs_pi) * advantages.detach()).sum()

In [4]:
def check_vol():
    return len(list(filter(lambda x: x.volatile, list(agent.parameters())))) == 0

def get_flat_grads_without_try(model, loss):
    """
    temporary, remove it later
    """
    
    grads = []
    for name, param in model.named_parameters():
        grads.append(torch.autograd.grad(loss, param, create_graph=True)[0])
            
    return torch.cat([grad.view(-1) for grad in grads])

In [3]:
cg_damping = 1e-3
max_kl = 1e-2

In [7]:
def learn(agent, env, optimizer, n_timesteps=1e5, gamma=0.99, lambda_gae=0.95, entr_coef=1e-3, log_interval=1e4):
    """
    Optimize networks parameters via interacting with env
    Arguments:
        agent           --  agent to optimize
        env             --  environment to interact with
        n_episodes      --  number of full interaction emulations, int
        lambda_gae      --  mixing coefficient in generalized advantage estimation
        entr_coef       --  entropy loss multiplier, float
        log_interval    --  number of timesteps to print debug info, int
    """

    agent.train()
    returns = []
    timestep = 0
    episode = 0

    while timestep < n_timesteps:
        s = env.reset()
        done = False
        ts = TrajStats()

        episode += 1
        while not done:
            logits, value = agent.forward(s)
            a = agent.sample_action(logits)
            s_new, r, done, _ = env.step(a)
            
            ts.append(r, F.log_softmax(logits, dim=-1)[a], value, logits, s, a)
            s = s_new
            timestep += 1
            if timestep % log_interval == 0:
                print('{} timesteps, av. return: {:.3f}'.format(timestep, np.mean(returns[-50:])))
        
        advantages = ts.calc_gaes(gamma, lambda_gae)
        episode_returns = ts.calc_episode_returns(gamma)
        logs_pi = ts.get_logs_pi_a()
        returns.append(ts.calc_return(gamma))

        #entropy      = -(aprobs_var * torch.exp(aprobs_var)).sum()
        #critic_loss  = 0.5*advantages.pow(2).sum()
        critic_loss  = 0.5*(ts.get_values() - episode_returns).pow(2).sum()
        
        optimizer.zero_grad()
        critic_loss.backward(retain_graph=True)
        
        actor_loss   = -(logs_pi * advantages.detach()).sum()  # minus added in order to ascend
        flat_grads = get_flat_grads(agent, actor_loss).data
        if np.allclose(flat_grads.numpy(), 0):
            print('zero gradients, passing')
            continue
        
        logits = ts.get_logits()
        kl = compute_kl(logits, logits.detach())
        flat_grads_kl = get_flat_grads(agent, kl)

        hess_vec = lambda vec: hess_vec_full(vec, agent, flat_grads_kl, cg_damping)
        
        stepdir = cg(hess_vec, -flat_grads)
        shs = 0.5 * torch.dot(stepdir, hess_vec(stepdir))
        lm = np.sqrt(shs / max_kl)
        proposed_step = stepdir / lm
        neggdotstepdir = torch.dot(-flat_grads, stepdir)
        
        compute_obj = lambda flat_params: compute_obj_full(flat_params, agent, ts, gamma, lambda_gae)
            
        params_prev = get_flat_params(agent)
        success, params_new = linesearch(compute_obj, params_prev, proposed_step, neggdotstepdir / lm)
        set_flat_params(agent, params_new)
        
        optimizer.step()

In [8]:
n_timesteps = 20000
gamma = 0.99
log_interval = 1000

env = gym.make('CartPole-v1')
#env = gym.make('FrozenLake-v0')
set_seeds(env, 417, False)

agent = AgentA2C(env)
optimizer = optim.Adam(agent.parameters())

learn(agent, env, optimizer, n_timesteps=n_timesteps, gamma=gamma, log_interval=log_interval)

1000 timesteps, av. return: 22.668
2000 timesteps, av. return: 26.327
3000 timesteps, av. return: 29.203
4000 timesteps, av. return: 30.477
5000 timesteps, av. return: 32.536
6000 timesteps, av. return: 35.113
7000 timesteps, av. return: 36.791
8000 timesteps, av. return: 35.596
9000 timesteps, av. return: 34.542
10000 timesteps, av. return: 32.623
11000 timesteps, av. return: 31.952
12000 timesteps, av. return: 33.327
13000 timesteps, av. return: 32.903
14000 timesteps, av. return: 32.478
15000 timesteps, av. return: 31.583
16000 timesteps, av. return: 31.910
17000 timesteps, av. return: 32.982
18000 timesteps, av. return: 31.997
19000 timesteps, av. return: 32.865
20000 timesteps, av. return: 32.405


In [ ]:
n_timesteps = 30000
gamma = 0.99
log_interval = 1000

#env = gym.make('CartPole-v1')
env = gym.make('FrozenLake-v0')
set_seeds(env, 417, False)

agent = AgentA2C(env)
optimizer = optim.Adam(agent.parameters())

learn(agent, env, optimizer, n_timesteps=n_timesteps, gamma=gamma, log_interval=log_interval)